# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# 6 - ALS Model - Reduced Data: Grid Search + Cross-Validation

This time, as explored in the EDA NB, let's consider removing customers who we feel have too few or too many purchases to influence the model in the intended way.

Examples:
- Too few - Customers who have only bought 1 comic (series).
- Too many - Customers with > 1000 series (for example, think all eBay customers are rolled into one account number).

# Libraries

In [13]:
%matplotlib inline
%load_ext autoreload
# %autoreload 1 #would be where you need to specify the files
# %aimport comic_recs

import pandas as pd # dataframes
import os
import pickle

# Data storage
from sqlalchemy import create_engine # SQL helper
#import psycopg2 as psql #PostgreSQL DBs

# import necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import DataFrame

# Custom
import lib.data_fcns as dfc
import lib.keys  # Custom keys lib
import lib.comic_recs as cr

import time
import itertools
from functools import reduce
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# instantiate SparkSession object
spark = pyspark.sql.SparkSession.builder.master("local[*]").getOrCreate()
# spark = SparkSession.builder.master("local").getOrCreate()

## Import Data

We've previously set aside the dataset into a `json` file.

In [3]:
# We have previously created a version of the transactions table and filtered it down.
sold = spark.read.json('raw_data/als_input_filtered.json')

In [4]:
# Persist the data
sold.persist()

DataFrame[account_id: bigint, bought: bigint, comic_id: bigint]

### ALS Model

Let's start with  train/test split.

In [82]:
# Split data into training and test set
(train, test) = sold.randomSplit([.8, .2], seed=41916)

Make sure shapes make sense.

In [83]:
print(train.count(), len(train.columns))

49402 3


In [84]:
print(test.count(), len(test.columns))

12469 3


In [85]:
# Evaluate the model by computing the RMSE on the test data
eval_reg = RegressionEvaluator(metricName="rmse"
                               , labelCol="bought"
                               , predictionCol="prediction")

### Grid Search

In [86]:
# hyper-param config
num_iterations = [10, 20]
ranks = [5, 10, 15, 20]
reg_params = [0.01, 0.1, 0.5]
alphas = [1, 50, 500, 1000, 5000]

Let's further subset into test and validation sets.

In [88]:
# Split data into training and validation sets
(gs_train, gs_val) = train.randomSplit([.8, .2], seed=41916)

In [89]:
# grid search and select best model
start_time = time.time()
#final_model, param_errors = tr_ALS(train, test, eval_reg, num_iterations, reg_params, ranks, alphas)
final_model, params_errs = cr.train_ALS(gs_train, gs_val, eval_reg, num_iterations, reg_params, ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

10 iterations, 5 latent factors, regularization=0.01, and alpha @ 1 : validation error is 0.8618
10 iterations, 5 latent factors, regularization=0.01, and alpha @ 50 : validation error is 0.4835
10 iterations, 5 latent factors, regularization=0.01, and alpha @ 500 : validation error is 0.3411
10 iterations, 5 latent factors, regularization=0.01, and alpha @ 1000 : validation error is 0.3285
10 iterations, 5 latent factors, regularization=0.01, and alpha @ 5000 : validation error is 0.3250
10 iterations, 5 latent factors, regularization=0.01, and alpha @ 10000 : validation error is 0.3208
10 iterations, 10 latent factors, regularization=0.01, and alpha @ 1 : validation error is 0.8451
10 iterations, 10 latent factors, regularization=0.01, and alpha @ 50 : validation error is 0.4952
10 iterations, 10 latent factors, regularization=0.01, and alpha @ 500 : validation error is 0.4145
10 iterations, 10 latent factors, regularization=0.01, and alpha @ 1000 : validation error is 0.4219
10 iter

20 iterations, 10 latent factors, regularization=0.01, and alpha @ 10000 : validation error is 0.4635
20 iterations, 15 latent factors, regularization=0.01, and alpha @ 1 : validation error is 0.8344
20 iterations, 15 latent factors, regularization=0.01, and alpha @ 50 : validation error is 0.5130
20 iterations, 15 latent factors, regularization=0.01, and alpha @ 500 : validation error is 0.4603
20 iterations, 15 latent factors, regularization=0.01, and alpha @ 1000 : validation error is 0.4735
20 iterations, 15 latent factors, regularization=0.01, and alpha @ 5000 : validation error is 0.5183
20 iterations, 15 latent factors, regularization=0.01, and alpha @ 10000 : validation error is 0.5257
20 iterations, 20 latent factors, regularization=0.01, and alpha @ 1 : validation error is 0.8314
20 iterations, 20 latent factors, regularization=0.01, and alpha @ 50 : validation error is 0.5423
20 iterations, 20 latent factors, regularization=0.01, and alpha @ 500 : validation error is 0.5244


Py4JJavaError: An error occurred while calling o28972.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69

Save the descriptive results

In [ ]:
param_errs_rd_1 = params_errs

In [ ]:
with open('support_data/params_errs_rd1.pkl', 'wb') as f:
    pickle.dump(param_errs_rd_1, f)
    
# Example - load pickle
# pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# pe1 = pickle.load(pickle_in)

Hmmm. Let's put `params_errs` into a dataframe and find the model with the lowest error!

In [ ]:
gs_cols = ['max_iters', 'reg', 'rank', 'alpha', 'rmse']

In [ ]:
gs_df = pd.DataFrame(params_errs, columns=gs_cols)

In [ ]:
gs_df.head()

In [ ]:
min_err = gs_df.rmse.min()

In [ ]:
gs_df.loc[gs_df['rmse']==min_err]

**OK**. Let's call it good. 

## Results 
Looks like the best parameters we could find are:
- `maxIter` = 20
- `rank` = 10
- `regParam` = 0.1 (default)
- `alpha` = 40

Let's do cross-validate this candidate model.

In [ ]:
best_max_iter = gs_df['max_iters']
best_reg = gs_df['reg']
best_rank = gs_df['rank']
best_alpha = gs_df['alpha']

## Cross Validation

Let's cross-validate because we didn't actually do it in the grid search. We want to make sure that the selected model is not overfitting.

The built-in cross validator in `Spark` keeps breaking when I try to use it, so let's build our own function.

In [ ]:
k = 5
random_seed = 41916

In [ ]:
folds = cr.get_spark_k_folds(sold, k=5, random_seed=random_seed)

In [ ]:
# Create ALS instance for cv with our chosen parametrs
als_cv = ALS(maxIter=best_max_iter,
          rank=best_rank,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          implicitPrefs=True,
          regParam=best_reg,
          alpha=best_alpha,
          coldStartStrategy='drop', # we want to drop so can get through CV
          seed=41916)

In [ ]:
errors = cr.get_cv_errors(folds, als_cv, eval_reg)

In [ ]:
# Make sure that # of errors = k
k == len(errors)

In [ ]:
print(errors)

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (errors.mean(), errors.std() * 2))

Looks good! Consistent with the best we've seen, and very stable.

## Test the Candidate Model

In [ ]:
# Create ALS instance and fit model
als = ALS(maxIter=best_max_iter,
          rank=best_rank,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          implicitPrefs=True,
          regParam=best_reg,
          alpha=best_alpha,
          coldStartStrategy='nan',
          seed=41916)
model_use = als.fit(train)

In [ ]:
# get predictions on test
test_preds = model.transform(test)

# Evaluate test
test_rmse = evaluator.evaluate(test_preds)
test_rmse

### Get Top N recommendations for Single User

Let's make a reference list of `account_id`'s, for testing purposes.

In [11]:
n_to_test = 2

users = (sold.select(als.getUserCol())
                          .sample(False
                                  ,n_to_test/sold.count()
                                  )
        )
users.persist()
users.show(2)

+----------+
|account_id|
+----------+
|       192|
|       569|
+----------+



We developed and wrote the functionality out to a function in `comic_recs.py`

###  Testing function!

- Pass the function to a pandas dataframe. 
- Function will ask for an account_id.
- Will return top n, n defined in parameters.

In [10]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=5)
top_n_df

192
Total Runtime: 28.51 seconds


,comic_title
1,Brightest Day (DC)
2,New Mutants (Marvel)
3,Fear Itself (Marvel)
4,Batman Incorporated (DC)
5,Birds of Prey (DC)


In [11]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=5)
top_n_df

569
Total Runtime: 29.68 seconds


,comic_title
1,Thor (Marvel)
2,Superior Spider-Man (Marvel)
3,Invincible (Image)
4,Image Firsts Saga Curr Ptg (Image)
5,Superior Iron Man (Marvel)


In [12]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=10)
top_n_df

161
Total Runtime: 21.43 seconds


,comic_title
1,Aliens Dead Orbit (Dark Horse)
2,Beef (Image)
3,Vinegar Teeth (Dark Horse)
4,Leviathan (Image)
5,Shaolin Cowboy Wholl Stop th (Dark Horse)
6,Head Lopper (Image)
7,Crossed Badlands (Avatar)
8,Punisher (Marvel)
9,Godzilla Oblivion (IDW)
10,Nova (Marvel)


## Conclusions
- Seems realistic? Only three tests, but the results seem 'individualized' in the sense that there is no overlap between the sets (albeit small samples).

## Save the Model!

In [13]:
model_use.save('als_use')

## Retrieving Saved Model

In [14]:
comic_rec_model = ALSModel.load('als_use')

In [15]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=comic_rec_model, topn=10)
top_n_df

161
Total Runtime: 5.66 seconds


,comic_title
1,Aliens Dead Orbit (Dark Horse)
2,Beef (Image)
3,Vinegar Teeth (Dark Horse)
4,Leviathan (Image)
5,Shaolin Cowboy Wholl Stop th (Dark Horse)
6,Head Lopper (Image)
7,Crossed Badlands (Avatar)
8,Punisher (Marvel)
9,Godzilla Oblivion (IDW)
10,Nova (Marvel)
